In [1]:
import pandas as pd
dataset = pd.read_csv('hate_speech.csv')
dataset.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [2]:
dataset.shape

(5242, 3)

In [3]:
dataset.label.value_counts()

,count
label,
0,3000
1,2242


In [4]:
for index, tweet in enumerate(dataset["tweet"][10:15]):
  print(index+1, "-", tweet)

1 -  â #ireland consumer price index (mom) climbed from previous 0.2% to 0.5% in may   #blog #silver #gold #forex
2 - we are so selfish. #orlando #standwithorlando #pulseshooting #orlandoshooting #biggerproblems #selfish #heabreaking   #values #love #
3 - i get to see my daddy today!!   #80days #gettingfed
4 - ouch...junior is angryð#got7 #junior #yugyoem   #omg 
5 - i am thankful for having a paner. #thankful #positive     


In [5]:
import re
def clean_text(text):
  text = re.sub(r'[^a-zA-Z]',' ', text)
  text = re.sub(r'[^\x00-\x7F]+', ' ', text)
  text = text.lower()
  return text

In [6]:
dataset['clean text'] = dataset.tweet.apply(lambda x: clean_text(x))

In [7]:
dataset.head(10)

,id,label,tweet,clean text
0,1,0,@user when a father is dysfunctional and is s...,user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...,user user thanks for lyft credit i can t us...
2,3,0,bihday your majesty,bihday your majesty
3,4,0,#model i love u take with u all the time in ...,model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation,factsguide society now motivation
5,6,0,[2/2] huge fan fare and big talking before the...,huge fan fare and big talking before the...
6,7,0,@user camping tomorrow @user @user @user @use...,user camping tomorrow user user user use...
7,8,0,the next school year is the year for exams.ð...,the next school year is the year for exams ...
8,9,0,we won!!! love the land!!! #allin #cavs #champ...,we won love the land allin cavs champ...
9,10,0,@user @user welcome here ! i'm it's so #gr...,user user welcome here i m it s so gr...


In [8]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
len(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


179

In [9]:
stop = stopwords.words('english')

In [10]:
def gen_freq(text):
  word_list = []
  for tw_words in text.split():
    word_list.extend(tw_words)
  word_freq = pd.Series(word_list).value_counts()
  word_freq = word_freq.drop(stop, errors='ignore')
  return word_freq

In [11]:
def any_neg(words):
  for word in words:
    if word in ['no', 'nor','non', 'not'] or re.search(r"\wn't", word):
      return 1
  else:
    return 0

In [12]:
def any_rare(words, rare_100):
  for word in words:
    if word in rare_100:
      return 1
    else:
      return 0

In [13]:
def is_question(words):
  for word in words:
    if word in ['when', 'what', 'how', 'why', 'who', 'where']:
      return 1
    else:
      return 0

In [14]:
dataset.head(10)

,id,label,tweet,clean text
0,1,0,@user when a father is dysfunctional and is s...,user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...,user user thanks for lyft credit i can t us...
2,3,0,bihday your majesty,bihday your majesty
3,4,0,#model i love u take with u all the time in ...,model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation,factsguide society now motivation
5,6,0,[2/2] huge fan fare and big talking before the...,huge fan fare and big talking before the...
6,7,0,@user camping tomorrow @user @user @user @use...,user camping tomorrow user user user use...
7,8,0,the next school year is the year for exams.ð...,the next school year is the year for exams ...
8,9,0,we won!!! love the land!!! #allin #cavs #champ...,we won love the land allin cavs champ...
9,10,0,@user @user welcome here ! i'm it's so #gr...,user user welcome here i m it s so gr...


In [17]:
dataset.columns

Index(['id', 'label', 'tweet', 'clean text'], dtype='object')

In [18]:
dataset['clean_text'] = dataset['tweet'].apply(lambda x: clean_text(x))

dataset['word_count'] = dataset['clean_text'].apply(lambda x: len(x.split()))
dataset['char_count'] = dataset['clean_text'].apply(lambda x: len(x))
dataset['any_neg'] = dataset['clean_text'].apply(lambda x: any_neg(x.split()))
dataset['is_question'] = dataset['clean_text'].apply(lambda x: is_question(x.split()))

rare_100 = gen_freq(' '.join(dataset['clean_text'])).tail(100).index.tolist()
dataset['any_rare'] = dataset['clean_text'].apply(lambda x: any_rare(x.split(), rare_100))

X = dataset[['word_count', 'any_neg', 'any_rare', 'char_count', 'is_question']]
y = dataset['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print("Train and test data split successful!")

Train and test data split successful!


In [19]:
from sklearn.naive_bayes import GaussianNB

model= GaussianNB()
model = model.fit(X_train,y_train)
pred= model.predict(X_test)


In [20]:
model.predict(X_test[5:10])

array([0, 0, 1, 1, 0])

In [21]:
from sklearn.metrics import accuracy_score
print("Accuracy:", accuracy_score(y_test,pred)*100, "%")

Accuracy: 54.33746425166825 %


In [22]:
from sklearn.ensemble import RandomForestClassifier
clf_rf = RandomForestClassifier()
clf_rf.fit(X_train, y_train)
rf_pred= clf_rf.predict(X_test).astype(int)


In [23]:
from sklearn.metrics import classification_report, confusion_matrix, \
accuracy_score
print(confusion_matrix(y_test,rf_pred))
print(classification_report(y_test,rf_pred))
print(accuracy_score(y_test,rf_pred))

[[419 180]
 [233 217]]
              precision    recall  f1-score   support

           0       0.64      0.70      0.67       599
           1       0.55      0.48      0.51       450

    accuracy                           0.61      1049
   macro avg       0.59      0.59      0.59      1049
weighted avg       0.60      0.61      0.60      1049

0.6062917063870352


In [26]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(class_weight='balanced')
logreg.fit(X_train, y_train)


LogisticRegression(class_weight='balanced')

In [27]:
y_pred = logreg.predict(X_test)